### Import Libraries

In [5]:
import pandas as pd
import scipy.io
from itertools import combinations_with_replacement as cwr

### Import data

In [6]:
mat = scipy.io.loadmat(r'C:\downloads\NIPS2008_train_data_matlab\remanei_size=25000_subsample=1.mat')
mat

{'LT': array([[-1., -1.,  1., ..., -1., -1., -1.]]),
 'XT': array([ 'ATGTTCAGTATATGGAGAAGTGAAAATTGAAGTTCACTTTGAGATCACTAAAATGAGCGGAGTTTTGGCAAGAAAAACGGTGAATTTTGATGAAAAAATGAAAGAATTCTCGGATGTTGTACTGGTGGTTGGAGAGGAGAA',
        'TGACAGAGGAGGTCGTGGCCGATCTAATGGAGCCCCACGCGAAAGTCGAGGAGAAAGCCGAGAACATAATCGTGAGACTCGTGATAATAAAGAAGGTGGAGCCCCTCGCCCAACCCGTGGCGGTTTCGAACGGCCCTACGT',
        'ATCGTTTAGATTTATTATAAAGATTGTTGTGTTTCGAATGATTGTTTGCAAACCTTTTAAAGAACGAGTCTCAACTTCTAGCCGCAGGAATTTTATCGACGTTCTGCCTGAACCGAAAAGTACTGAAATTCCTTCCATTGA',
        ...,
        'GTCGGAAAACAAAACAATTATCGATTTTTGTGGTTCTCGCACTCATTTTTCTCTCTTTTCAGTCAAATTACTCGAATTGAAAACAGAAAACAAAACACTATGAAAAGACTGCAAGCTAGCCAAAATAATTATCGATTTTTG',
        'CTCGCTCCACCCTGCGTTTCCTCTCAATACTTCTTGAAACAGTGTTGTGCCACGAGATGAAGTGTGGACATGTGAAGGTAGCGGTAATATTTTTGCCCGTGCACAAGTTTGTCTTCTCTTGTTGAAATGTTTTTCATATGT',
        'GTTCTTACAACTGCGCTCCACCGAAATGCCCTTGAAAATGTCTCTTTTTCTCTGAGTCTCAGAATTTCGCGCACTATTTTCTCCGGTTCCGGTAGAGCGCGGTTGTAAGATGGGTGTCGTGCTAGTATATGCTATAACTCG'],
       d

In [7]:
len(list(mat['LT'][0]))

25000

Final Data Set

In [8]:
data=pd.DataFrame(list(zip(list(mat['LT'][0]),list(mat['XT']))),columns=['label','DNA'])
data.head()

,label,DNA
0,-1.0,ATGTTCAGTATATGGAGAAGTGAAAATTGAAGTTCACTTTGAGATC...
1,-1.0,TGACAGAGGAGGTCGTGGCCGATCTAATGGAGCCCCACGCGAAAGT...
2,1.0,ATCGTTTAGATTTATTATAAAGATTGTTGTGTTTCGAATGATTGTT...
3,-1.0,GCGTCGACAGACTCGTCGAGTACGGTAAGTACAGTAAGACACTCTT...
4,-1.0,ACGATTCTTCAATCGATCGAAAGGAACTGCCAGACACTCTGGTTTC...


### Feature Extraction

Amino acids in the c.remanei species

In [9]:
aminos='ACGT'

In [10]:
from itertools import combinations_with_replacement as cwr
import itertools

All the combination of trimers possible with amino acids in c.remanei

In [11]:
combs = [''.join(p) for p in list(itertools.product(aminos,repeat= 3))]
len(combs)

64

In [12]:
columns=[]
for i in range(141):
    for a in aminos:
        columns=columns+['p'+str(i+1)+'_'+a]
columns=columns+[s+'_bef' for s in combs]+[s+'_aft' for s in combs]
columns+=aminos
len(columns)

696

columns contain the dummy variable names for the length of RNA and position of trimers

In [13]:
import numpy as np
feat=np.zeros((25000,len(columns)))

Creating features from RNA strings

In [14]:
for j in range(25000):
    for i in range(69):
        feat[j,563+combs.index(data.DNA[j][i:i+3])]+=1
        feat[j,i*4+aminos.index(data.DNA[j][i])]=1
    for i in range(69,139):
        feat[j,563+64+combs.index(data.DNA[j][i:i+3])]+=1
        feat[j,i*4+aminos.index(data.DNA[j][i])]=1
    for i in range(139,141):
        feat[j,i*4+aminos.index(data.DNA[j][i])]=1

Feature matrix

In [15]:
feat_mat=pd.DataFrame(feat,columns=columns)
feat_mat.head()

,p1_A,p1_C,p1_G,p1_T,p2_A,p2_C,p2_G,p2_T,p3_A,p3_C,...,TGG_aft,TGT_aft,TTA_aft,TTC_aft,TTG_aft,TTT_aft,A,C,G,T
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,2.0,0.0,1.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,1.0,4.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,1.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,1.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0


In [16]:
feat_mat['label']=data.label
feat_mat.columns

Index(['p1_A', 'p1_C', 'p1_G', 'p1_T', 'p2_A', 'p2_C', 'p2_G', 'p2_T', 'p3_A',
       'p3_C',
       ...
       'TGT_aft', 'TTA_aft', 'TTC_aft', 'TTG_aft', 'TTT_aft', 'A', 'C', 'G',
       'T', 'label'],
      dtype='object', length=697)

### Feature Selection

In [17]:
from sklearn.linear_model import Lasso
from sklearn.svm import LinearSVC
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel

In [18]:
lsvc = LinearSVC(C=0.4, penalty="l1", dual=False).fit(feat_mat[columns], feat_mat.label)
model = SelectFromModel(lsvc, prefit=True)
X_new = model.transform(feat_mat[columns])
X_new.shape

(25000, 295)

In [19]:
from itertools import compress
feat_filtered=list(compress(columns,model.get_support()))

In [20]:
X_new=pd.DataFrame(X_new)
X_new.columns=feat_filtered
X_new.columns

Index(['p1_A', 'p1_T', 'p2_G', 'p3_A', 'p3_G', 'p4_C', 'p4_G', 'p5_A', 'p7_A',
       'p8_C',
       ...
       'TAC_aft', 'TAG_aft', 'TAT_aft', 'TCA_aft', 'TCG_aft', 'TGA_aft',
       'TGC_aft', 'TGG_aft', 'TTC_aft', 'TTG_aft'],
      dtype='object', length=295)

Above are the columns selected through L1 penalty

### Training - Testing data split

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
x_train, x_val, y_train, y_val = train_test_split(X_new,feat_mat.label,
                                                  test_size = .1,
                                                  random_state=12)

In [32]:
len(x_train),len(x_val)

(22500, 2500)

### Over Sampling

In [3]:
from imblearn.over_sampling import SMOTE

In [76]:
r={1:500}
smote=SMOTE(random_state=42,ratio=r,)
x_train_res, y_train_res = smote.fit_sample(x_train, y_train)

In [81]:
x_val_res, y_val_res = smote.fit_sample(x_val, y_val)

In [73]:
len(x_train_res)

24294

### Modeling

In [65]:
import statsmodels.api as sm

In [77]:
y_train_res[y_train_res==-1]=0

In [79]:
from sklearn import svm

In [84]:
clf.fit(x_train_res)

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.1, kernel='rbf',
      max_iter=-1, nu=0.1, random_state=None, shrinking=True, tol=0.001,
      verbose=False)

In [86]:
clf = svm.OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1)

In [89]:
clf.fit(x_train_res)

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.1, kernel='rbf',
      max_iter=-1, nu=0.1, random_state=None, shrinking=True, tol=0.001,
      verbose=False)

In [90]:
y_pred_train = clf.predict(x_train_res)
y_pred_test = clf.predict(x_val_res)

In [95]:
sum(y_pred_test==y_val_res)/len(y_val_res)

0.83165322580645162